In [1]:
from datetime import datetime

from tensorflow.keras.activations import relu, sigmoid, tanh, swish
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPool2D, UpSampling2D, InputLayer, Lambda, Dense
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers
import glob
import math
import numpy as np
import os
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp

2024-08-16 11:12:47.351775: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 11:12:47.351862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 11:12:47.490490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 11:12:47.846347: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-16 11:12:49.946561: W tensorflow/compiler/tf2

In [2]:
import tensorflow as tf

In [18]:
def Conv_Pool_block(x0,nfilters,w1=3,w2=3,p1=2,p2=2, padding='same', data_format='channels_first'):
    x0 = Conv2D(nfilters, (w1, w2), activation='relu', padding=padding, data_format=data_format)(x0)
    x0 = Conv2D(nfilters, (w1, w2), activation='relu', padding=padding, data_format=data_format)(x0)
    x0 = MaxPool2D((p1, p2), padding='same', data_format=data_format)(x0)
    return x0

In [4]:
#Create a test tensor in tensorflow of size (10,4,64,64)

test_tensor = tf.random.normal(shape=(10, 4, 64, 64))




2024-08-06 16:44:32.162364: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-06 16:44:33.055640: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-06 16:44:33.055741: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-06 16:44:33.080449: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-06 16:44:33.080521: I external/local_xla/xla/stream_executor

In [19]:
#Feed through Conv_Pool_Block
res = Conv_Pool_block(test_tensor,4)

In [20]:
res.shape

TensorShape([10, 4, 32, 32])

Testing out poisson likelihood shapes

In [4]:
tfpl = tfp.layers
tfd = tfp.distributions

In [5]:
dist_poisson_intensity = tfpl.DistributionLambda(lambda amplitude:
                                       (tfd.Independent(
                                           tfd.Poisson(
                                               (amplitude**2)))))

In [6]:
#Make random tf tensor of integers of shape (10,4,64,64)
amp_tensor = tf.random.uniform(shape=(10,4,64,64), minval=1, maxval=200)

pred_sampled = dist_poisson_intensity(amp_tensor)

2024-08-16 11:19:02.290938: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-16 11:19:03.589007: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-16 11:19:03.589071: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-16 11:19:03.610209: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-16 11:19:03.610316: I external/local_xla/xla/stream_executor

Instructions for updating:
Please pass an integer value for `reinterpreted_batch_ndims`. The current behavior corresponds to `reinterpreted_batch_ndims=tf.size(distribution.batch_shape_tensor()) - 1`.


In [7]:
observed_tensor = tf.random.uniform(shape=(10,4,64,64), minval=1, maxval=200)
Negloglik = pred_sampled.log_prob(observed_tensor)

In [9]:
Negloglik

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-2.1338213e+08, -2.0952843e+08, -2.1422430e+08, -2.1267229e+08,
       -2.1201318e+08, -2.1082531e+08, -2.1294994e+08, -2.1142658e+08,
       -2.1151280e+08, -2.1140445e+08], dtype=float32)>